In [3]:
import glob
import os
import pandas as pd

# Set the directory where the CSV files are located
dir_path = "."

# Get all the CSV files in the directory
all_files = glob.glob(os.path.join(dir_path, "*.csv"))

# Combine the CSV files into a single DataFrame
df = pd.concat((pd.read_csv(f) for f in all_files))

# Save the combined DataFrame to a new CSV file
df.to_csv("combined.csv", index=False)

In [5]:
df = pd.read_csv('combined.csv')
df

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
0,Theft from Vehicle,2022,1,29,9,30,19XX W GEORGIA ST,Central Business District,490079.3981,5.460210e+06
1,Theft from Vehicle,2022,1,29,13,30,19XX W GEORGIA ST,Central Business District,490079.3981,5.460210e+06
2,Theft from Vehicle,2022,2,4,13,0,19XX W GEORGIA ST,Central Business District,490079.3981,5.460210e+06
3,Theft from Vehicle,2022,2,20,14,0,19XX W GEORGIA ST,Central Business District,490079.3981,5.460210e+06
4,Theft from Vehicle,2022,4,5,13,0,19XX W GEORGIA ST,Central Business District,490079.3981,5.460210e+06
...,...,...,...,...,...,...,...,...,...,...
55393,Theft from Vehicle,2022,2,8,16,45,17XX GLEN DR,Strathcona,494082.1971,5.457439e+06
55394,Theft from Vehicle,2022,3,8,22,40,17XX GLEN DR,Strathcona,494082.1971,5.457439e+06
55395,Theft from Vehicle,2022,5,1,0,0,17XX GLEN DR,Strathcona,494082.6659,5.457406e+06
55396,Theft from Vehicle,2022,5,17,0,18,18XX CLARK DR,Strathcona,494366.0040,5.457253e+06
